In [1]:
import pandas as pd
import numpy as np
import datetime as dt

from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split


In [2]:
#Cargo los archivos de entrenamiento
eventos = pd.read_csv("../events_up_to_01062018.csv")
labels = pd.read_csv("../labels_training_set.csv")

C:\Users\felip\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
labels.sample(20)

,person,label
10008,e9521bde,0
342,ac4a94ea,1
10823,f52df323,0
1755,50eeb193,0
15190,42c493db,0
16530,2062d6ab,0
2892,bceebdd1,0
12822,db9e8233,0
14840,a2c9bda9,0
2430,1520f8cf,0


In [4]:
#Veo las columnas de eventos 
list(eventos)

['timestamp',
 'event',
 'person',
 'url',
 'sku',
 'model',
 'condition',
 'storage',
 'color',
 'skus',
 'search_term',
 'staticpage',
 'campaign_source',
 'search_engine',
 'channel',
 'new_vs_returning',
 'city',
 'region',
 'country',
 'device_type',
 'screen_resolution',
 'operating_system_version',
 'browser_version']

In [93]:
#Me quedo con los set de datos que coincidan con los labels y sean inferiores al 01/06/2018
set_entrenamiento = pd.merge(eventos, labels, on='person', how='inner')
set_entrenamiento.sample(20)

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,label
917146,2018-05-16 06:21:26,viewed product,26858144,NaN,214.0,iPhone 5c,Excelente,16GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
156476,2018-04-24 23:41:19,viewed product,e3407832,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7232,2018-05-27 18:02:37,viewed product,eb6c498c,NaN,6847.0,iPhone 6S,Bom,16GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
815187,2018-05-31 16:56:52,viewed product,1ee7658e,NaN,8951.0,iPhone 6S,Muito Bom,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
718247,2018-04-17 15:29:58,viewed product,fe469427,NaN,14734.0,Samsung Galaxy A7 2017,Novo,32GB,Preto,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
788810,2018-05-22 16:03:37,search engine hit,83a2764c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
525870,2018-05-31 22:00:54,viewed product,56c5408b,NaN,2928.0,Motorola Moto MAXX,Muito Bom,64GB,Preto,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1100373,2018-05-23 14:52:11,search engine hit,3bad42d9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
693206,2018-05-13 23:06:39,viewed product,9caf8a87,NaN,4957.0,Motorola Moto G2 4G Dual,Excelente,16GB,Preto,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
460310,2018-05-04 16:54:02,ad campaign hit,d9e83eae,/comprar/samsung/galaxy-s5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [94]:
#Dropeo columnas que son irrelevantes

set_entrenamiento = set_entrenamiento.loc[:,['event','person','model','condition','storage','color',
                                          'search_term','campaing_source','channel','new_vs_returning','label']]
set_entrenamiento.sample(20)


C:\Users\felip\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,event,person,model,condition,storage,color,search_term,campaing_source,channel,new_vs_returning,label
166123,viewed product,c592b022,Samsung Galaxy S6 Edge Plus,Muito Bom,32GB,Preto,NaN,NaN,NaN,NaN,0
495973,generic listing,e28fabe1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
470807,brand listing,71505b42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
624822,viewed product,ffed0342,Samsung Galaxy A7 2017,Bom,32GB,Dourado,NaN,NaN,NaN,NaN,0
909017,searched products,7cc0f29b,NaN,NaN,NaN,NaN,5s 32,NaN,NaN,NaN,0
983007,viewed product,e13b4e5b,Samsung Galaxy A5,Excelente,16GB,Dourado,NaN,NaN,NaN,NaN,0
1027976,generic listing,17aacb23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
875767,viewed product,88da94b8,Samsung Galaxy S7 Edge,Bom,32GB,Dourado,NaN,NaN,NaN,NaN,1
1070804,viewed product,0887367d,Samsung Galaxy A7 2017,Excelente,32GB,Rosa,NaN,NaN,NaN,NaN,0
162995,brand listing,76712d3f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [95]:
#Agrego columna marca modelo
set_entrenamiento['marca'] = set_entrenamiento['model'].str.split(' ').str[0]
set_entrenamiento['modelo'] = set_entrenamiento['model'].str.split(' ').str[1].fillna('') + ' ' + set_entrenamiento['model'].str.split(' ').str[2].fillna('')


In [96]:
#Borro columna model
set_entrenamiento.drop(['model'], axis=1)
set_entrenamiento.sample(10)

,event,person,model,condition,storage,color,search_term,campaing_source,channel,new_vs_returning,label,marca,modelo
142418,brand listing,e40e146d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,
124741,viewed product,65a5e3d0,iPhone 5c,Bom,16GB,Amarelo,NaN,NaN,NaN,NaN,0,iPhone,5c
820325,viewed product,94e6ebcc,Sony Xperia Z3 Dual,Excelente,16GB,Branco,NaN,NaN,NaN,NaN,0,Sony,Xperia Z3
195019,viewed product,071e79e1,Motorola Moto X Play 4G Dual,Bom,32GB,Framboesa,NaN,NaN,NaN,NaN,1,Motorola,Moto X
991533,viewed product,c3c6ed89,iPhone 7 Plus,Bom,128GB,Dourado,NaN,NaN,NaN,NaN,0,iPhone,7 Plus
920712,viewed product,add442ab,Samsung Galaxy A5,Bom,16GB,Prata,NaN,NaN,NaN,NaN,0,Samsung,Galaxy A5
268624,viewed product,af076583,Samsung Galaxy S7,Bom,32GB,Rosa,NaN,NaN,NaN,NaN,0,Samsung,Galaxy S7
666244,visited site,ee6aecb8,NaN,NaN,NaN,NaN,NaN,NaN,Referral,Returning,0,NaN,
6611,viewed product,16d0e27d,Samsung Galaxy S5 Mini Duos,Excelente,16GB,Branco,NaN,NaN,NaN,NaN,0,Samsung,Galaxy S5
1065670,brand listing,bc030a03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,


In [98]:
#Agrego columna proviene de un explorador
set_entrenamiento['proviene_de_un_buscador'] =  np.where(set_entrenamiento['event'] =='search engine hit', 1, 0)
set_entrenamiento.sample(20)
#set_entrenamiento.set_index("index")

,event,person,model,condition,storage,color,search_term,campaing_source,channel,new_vs_returning,label,marca,modelo,proviene_de_un_buscador
885062,viewed product,1775ba85,Samsung Galaxy J7,Muito Bom,16GB,Branco,NaN,NaN,NaN,NaN,0,Samsung,Galaxy J7,0
340173,checkout,95af227d,Samsung Galaxy S6 Edge,Bom,32GB,Branco,NaN,NaN,NaN,NaN,0,Samsung,Galaxy S6,0
450211,generic listing,39ef79bf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,,0
1062628,viewed product,eb5145cd,Samsung Galaxy S3 i9300,Bom,16GB,Azul,NaN,NaN,NaN,NaN,0,Samsung,Galaxy S3,0
750632,search engine hit,6188fb1f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,,1
507417,viewed product,b30de517,Motorola Moto G5 Plus,Excelente,32GB,Ouro,NaN,NaN,NaN,NaN,0,Motorola,Moto G5,0
1133849,searched products,c1cbbc11,NaN,NaN,NaN,NaN,iPhone 5c 64 gb,NaN,NaN,NaN,0,NaN,,0
1005334,searched products,0299c90c,NaN,NaN,NaN,NaN,samsung j7,NaN,NaN,NaN,0,NaN,,0
1017014,viewed product,e8655740,Motorola Moto G5,Excelente,32GB,Platinum,NaN,NaN,NaN,NaN,0,Motorola,Moto G5,0
1049920,brand listing,00fa4cd7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,,0


In [99]:
#df['Data3'].groupby(df['Date']).transform('sum')
#Agrupamos por persona e identificamos si alguna vez vino de un explorador
set_entrenamiento['promedio'] = set_entrenamiento['proviene_de_un_buscador'].groupby(set_entrenamiento['person']).transform('mean')
set_entrenamiento.sample(10)
#set_entrenamiento.groupby('person').mean()

,event,person,model,condition,storage,color,search_term,campaing_source,channel,new_vs_returning,label,marca,modelo,proviene_de_un_buscador,promedio
883840,viewed product,e2f90506,Samsung Galaxy A5 2017,Muito Bom,32GB,Preto,NaN,NaN,NaN,NaN,0,Samsung,Galaxy A5,0,0.000000
246177,brand listing,9b1ba18d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,,0,0.100000
559707,visited site,afdedda5,NaN,NaN,NaN,NaN,NaN,NaN,Paid,Returning,1,NaN,,0,0.013889
87856,searched products,d77f3950,NaN,NaN,NaN,NaN,J7 prime,NaN,NaN,NaN,0,NaN,,0,0.000000
6762,generic listing,16d0e27d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,,0,0.002653
146387,viewed product,d6b69511,iPhone 4S,Bom,16GB,Preto,NaN,NaN,NaN,NaN,0,iPhone,4S,0,0.076087
923641,visited site,e0c3abed,NaN,NaN,NaN,NaN,NaN,NaN,Organic,Returning,0,NaN,,0,0.069767
1014399,viewed product,b1422615,Motorola Moto X2,Bom,32GB,Couro Vintage,NaN,NaN,NaN,NaN,0,Motorola,Moto X2,0,0.000000
1169997,viewed product,145fc7c1,Samsung Galaxy A7,Bom,16GB,Branco,NaN,NaN,NaN,NaN,0,Samsung,Galaxy A7,0,0.078947
344263,generic listing,73f9db35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,,0,0.067050


In [100]:
#Mapeo el valor si el promedio es mayor a cero
set_entrenamiento['search_engine_hit'] =  np.where(set_entrenamiento['promedio'] != 0, 1, 0)
set_entrenamiento.sample(10)


,event,person,model,condition,storage,color,search_term,campaing_source,channel,new_vs_returning,label,marca,modelo,proviene_de_un_buscador,promedio,search_engine_hit
3263,viewed product,16fad36a,iPhone 5,Bom,16GB,Preto,NaN,NaN,NaN,NaN,0,iPhone,5,0,0.028031,1
1091001,brand listing,4e8d56df,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,,0,0.016260,1
233314,generic listing,40fec7e8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,,0,0.047619,1
989900,ad campaign hit,517a7a96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,,0,0.011628,1
195859,viewed product,1dea6f79,Motorola Moto G5 Plus,Excelente,32GB,Platinum,NaN,NaN,NaN,NaN,0,Motorola,Moto G5,0,0.024324,1
1118939,viewed product,0d5f66ed,iPhone 5s,Excelente,64GB,Prateado,NaN,NaN,NaN,NaN,0,iPhone,5s,0,0.000806,1
180151,ad campaign hit,9b3b43aa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,,0,0.002405,1
1046930,viewed product,9886cd72,Motorola Moto G2 3G Dual,Muito Bom,8GB,Preto,NaN,NaN,NaN,NaN,1,Motorola,Moto G2,0,0.000000,0
208276,search engine hit,570029e3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,,1,0.137143,1
886048,viewed product,8f6c215e,iPhone 5c,Bom,8GB,Branco,NaN,NaN,NaN,NaN,1,iPhone,5c,0,0.047619,1


In [116]:
#Dropeo columnas promedio, proviene de un explorador y elimino registros con el evento search engine hit
set_entrenamiento = set_entrenamiento.loc[set_entrenamiento.event != 'search_engine_hit',]
set_entrenamiento.sample(10)


,event,person,model,condition,storage,color,search_term,campaing_source,channel,new_vs_returning,label,marca,modelo,proviene_de_un_buscador,promedio,search_engine_hit,proviene_de_un_camapania
734246,viewed product,9f31c8d8,iPhone 6,Muito Bom,64GB,Prateado,NaN,NaN,NaN,NaN,0,iPhone,6,0,0.000000,0,0
982480,generic listing,5c744844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,,0,0.009859,1,0
201375,brand listing,2bb8a9df,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,,0,0.128713,1,0
675925,visited site,06989c88,NaN,NaN,NaN,NaN,NaN,NaN,Paid,Returning,0,NaN,,0,0.002410,1,0
59405,viewed product,b3c45e4b,Quantum YOU,Excelente,32GB,Azul,NaN,NaN,NaN,NaN,1,Quantum,YOU,0,0.004511,1,0
349870,visited site,1321c62f,NaN,NaN,NaN,NaN,NaN,NaN,Direct,Returning,0,NaN,,0,0.009285,1,0
940142,search engine hit,52a4f0b6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,,1,0.002725,1,0
33616,ad campaign hit,94e7818d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,,0,0.002381,1,1
715171,viewed product,669134a2,Motorola Moto G4 Play DTV,Excelente,16GB,Cabernet,NaN,NaN,NaN,NaN,0,Motorola,Moto G4,0,0.013986,1,0
816386,visited site,64e0e51e,NaN,NaN,NaN,NaN,NaN,NaN,Paid,Returning,0,NaN,,0,0.084906,1,0


In [124]:
#Borro columnas
del set_entrenamiento['promedio']
del set_entrenamiento['proviene_de_un_buscador']
set_entrenamiento.head()

,event,person,model,condition,storage,color,search_term,campaing_source,channel,new_vs_returning,label,marca,modelo,search_engine_hit,proviene_de_un_camapania
0,viewed product,ad93850f,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,NaN,NaN,NaN,0,iPhone,5s,1,0
1,viewed product,ad93850f,iPhone 5s,Muito Bom,64GB,Prateado,NaN,NaN,NaN,NaN,0,iPhone,5s,1,0
2,viewed product,ad93850f,iPhone 5s,Bom - Sem Touch ID,16GB,Cinza espacial,NaN,NaN,NaN,NaN,0,iPhone,5s,1,0
3,viewed product,ad93850f,iPhone 5s,Bom - Sem Touch ID,16GB,Dourado,NaN,NaN,NaN,NaN,0,iPhone,5s,1,0
4,ad campaign hit,ad93850f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,,1,1


In [130]:
#Eliminamos eventos como generic listing
set_entrenamiento2 = set_entrenamiento.loc[set_entrenamiento.event != 'brand_listing',]
set_entrenamiento2 = set_entrenamiento2.loc[set_entrenamiento2.event != 'generic_listing',]
set_entrenamiento2 = set_entrenamiento2.loc[set_entrenamiento2.event != 'staticpage',]
set_entrenamiento2 = set_entrenamiento2.loc[set_entrenamiento2.event != 'conversion',]

set_entrenamiento2.head()

,event,person,model,condition,storage,color,search_term,campaing_source,channel,new_vs_returning,label,marca,modelo,search_engine_hit,proviene_de_un_camapania,promedio,ad_campaign_hit
0,viewed product,ad93850f,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,NaN,NaN,NaN,0,iPhone,5s,1,0,0.153846,True
1,viewed product,ad93850f,iPhone 5s,Muito Bom,64GB,Prateado,NaN,NaN,NaN,NaN,0,iPhone,5s,1,0,0.153846,True
2,viewed product,ad93850f,iPhone 5s,Bom - Sem Touch ID,16GB,Cinza espacial,NaN,NaN,NaN,NaN,0,iPhone,5s,1,0,0.153846,True
3,viewed product,ad93850f,iPhone 5s,Bom - Sem Touch ID,16GB,Dourado,NaN,NaN,NaN,NaN,0,iPhone,5s,1,0,0.153846,True
4,ad campaign hit,ad93850f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,,1,1,0.153846,True


In [132]:
#Agrego un campo si alguna vez vino de una campaña publicitaria
set_entrenamiento2['proviene_de_un_camapania'] =  np.where(set_entrenamiento2['event'] =='ad campaign hit', 1, 0)
set_entrenamiento2['promedio'] = set_entrenamiento2['proviene_de_un_camapania'].groupby(set_entrenamiento2['person']).transform('mean')
set_entrenamiento2['ad_campaign_hit'] =  np.where(set_entrenamiento2['promedio'] != 0, True, False)
del set_entrenamiento2['promedio']
del set_entrenamiento2['proviene_de_un_camapania']
set_entrenamiento2.sample(10)


,event,person,model,condition,storage,color,search_term,campaing_source,channel,new_vs_returning,label,marca,modelo,search_engine_hit,ad_campaign_hit
506369,ad campaign hit,28d5cb33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,,0,True
592185,visited site,78bdc823,NaN,NaN,NaN,NaN,NaN,NaN,Paid,Returning,0,NaN,,0,True
631345,viewed product,656c18ef,iPhone 6,Bom,64GB,Dourado,NaN,NaN,NaN,NaN,0,iPhone,6,0,True
482782,brand listing,a937984d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,,1,True
11468,ad campaign hit,99c11fb0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,,1,True
571120,viewed product,328bf1e2,Motorola Moto G3 4G,Bom,16GB,Preto,NaN,NaN,NaN,NaN,0,Motorola,Moto G3,1,True
927848,viewed product,b563761d,iPhone 5s,Muito Bom,32GB,Prateado,NaN,NaN,NaN,NaN,0,iPhone,5s,1,True
443714,visited site,75376ab4,NaN,NaN,NaN,NaN,NaN,NaN,Paid,Returning,0,NaN,,1,True
857117,viewed product,09487355,Samsung Galaxy Note 4,Muito Bom,32GB,Branco,NaN,NaN,NaN,NaN,0,Samsung,Galaxy Note,1,True
67061,search engine hit,056e7674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,,1,True


In [51]:
set_entrenamiento.head()

,timestamp,event,url,sku,model,condition,storage,color,skus,search_term,...,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,label
person,,,,,,,,,,,,,,,,,,,,,
ad93850f,2018-05-18 00:11:27,viewed product,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
ad93850f,2018-05-18 00:23:33,viewed product,NaN,318.0,iPhone 5s,Muito Bom,64GB,Prateado,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
ad93850f,2018-05-18 00:16:10,viewed product,NaN,5907.0,iPhone 5s,Bom - Sem Touch ID,16GB,Cinza espacial,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
ad93850f,2018-05-18 00:14:55,viewed product,NaN,6023.0,iPhone 5s,Bom - Sem Touch ID,16GB,Dourado,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
ad93850f,2018-05-18 00:11:26,ad campaign hit,/comprar/iphone/iphone-5s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [52]:
#Armamos arrays con los features que vamos a tener en cuenta
features=list(set_entrenamiento.columns)
features

['timestamp',
 'event',
 'url',
 'sku',
 'model',
 'condition',
 'storage',
 'color',
 'skus',
 'search_term',
 'staticpage',
 'campaign_source',
 'search_engine',
 'channel',
 'new_vs_returning',
 'city',
 'region',
 'country',
 'device_type',
 'screen_resolution',
 'operating_system_version',
 'browser_version',
 'label']

In [54]:
#Eliminamos algunos features que consideramos irrelevantes
#features.remove('person')
features.remove('timestamp')
features.remove('url')
features.remove('skus')
features.remove('sku')
features.remove('staticpage')
features.remove('search_engine')
features.remove('city')
features.remove('region')
features.remove('country')
features.remove('device_type')
features.remove('screen_resolution')
features.remove('operating_system_version')
features.remove('browser_version')
features.remove('label')



In [46]:
eventos.new_vs_returning.unique()

array([nan, 'Returning', 'New'], dtype=object)

In [55]:
#Lista de Features
features

['event',
 'model',
 'condition',
 'storage',
 'color',
 'search_term',
 'campaign_source',
 'channel',
 'new_vs_returning']

In [56]:
#Dividimos el set de entrenamientos para usar para la construccion del modelo y validarlo
train, validation=train_test_split(set_entrenamiento,test_size=0.10)
print("Tamaño set entrenamiento: ", len(train))
print("Tamaño set de validacion: ", len(validation))

Tamaño set entrenamiento:  1054697
Tamaño set de validacion:  117189


In [57]:
#Encodeamos nuestras features
lb_make = LabelEncoder()

train['event'] = lb_make.fit_transform(train['event'].astype(str))
train['model'] = lb_make.fit_transform(train['model'].astype(str))
train['condition'] = lb_make.fit_transform(train['condition'].astype(str))
train['color'] = lb_make.fit_transform(train['color'].astype(str))
train['search_term'] = lb_make.fit_transform(train['search_term'].astype(str))
train['campaign_source'] = lb_make.fit_transform(train['campaign_source'].astype(str))
train['channel'] = lb_make.fit_transform(train['channel'].astype(str))
train['storage'] = lb_make.fit_transform(train['storage'].astype(str))
train['new_vs_returning'] = lb_make.fit_transform(train['new_vs_returning'].astype(str))
#train['city'] = lb_make.fit_transform(train['city'].astype(str))
#train['region'] = lb_make.fit_transform(train['region'].astype(str))
#train['country'] = lb_make.fit_transform(train['country'].astype(str))
#train['device_type'] = lb_make.fit_transform(train['device_type'].astype(str))
#train['screen_resolution'] = lb_make.fit_transform(train['screen_resolution'].astype(str))
#train['operating_system_version'] = lb_make.fit_transform(train['operating_system_version'].astype(str))

validation['event'] = lb_make.fit_transform(validation['event'].astype(str))
validation['model'] = lb_make.fit_transform(validation['model'].astype(str))
validation['condition'] = lb_make.fit_transform(validation['condition'].astype(str))
validation['color'] = lb_make.fit_transform(validation['color'].astype(str))
validation['search_term'] = lb_make.fit_transform(validation['search_term'].astype(str))
validation['campaign_source'] = lb_make.fit_transform(validation['campaign_source'].astype(str))
validation['channel'] = lb_make.fit_transform(validation['channel'].astype(str))
validation['storage'] = lb_make.fit_transform(validation['storage'].astype(str))
validation['new_vs_returning'] = lb_make.fit_transform(validation['new_vs_returning'].astype(str))
#validation['city'] = lb_make.fit_transform(validation['city'].astype(str))
#validation['region'] = lb_make.fit_transform(validation['region'].astype(str))
#validation['country'] = lb_make.fit_transform(validation['country'].astype(str))
#validation['device_type'] = lb_make.fit_transform(validation['device_type'].astype(str))
#validation['screen_resolution'] = lb_make.fit_transform(validation['screen_resolution'].astype(str))
#validation['operating_system_version'] = lb_make.fit_transform(validation['operating_system_version'].astype(str))

C:\Users\felip\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\felip\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\felip\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.

In [58]:
#Empezamos a definir la variables x_train y_train 
x_train=train[features]
y_train=train['label']
x_validation = validation[features]
y_validation = validation['label']



In [ ]:
#Validamos nuestro modelo
model = KNeighborsClassifier(n_neighbors=20, n_jobs=-1)
model.fit(x_train, y_train)

In [15]:
#Hacemos un score del modelo

score = model.score(x_validation, y_validation)*100
print(score)

#predicted = model.predict(x_validation)
#print('Finalize')

91.52224184863768


In [16]:
#Nos traemos el set de datos de para predecir
#Leo el set de datos para predecir
kaggle_test = pd.read_csv("../trocafone_kaggle_test.csv");
kaggle_test.head()

,person
0,4886f805
1,0297fc1e
2,2d681dd8
3,cccea85e
4,4c8a8b93


In [17]:
test = pd.merge(kaggle_test,eventos, on='person', how='inner')
test.set_index('person', inplace=True)
test.head()

,timestamp,event,url,sku,model,condition,storage,color,skus,search_term,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
person,,,,,,,,,,,,,,,,,,,,,
4886f805,2018-05-18 00:11:59,viewed product,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4886f805,2018-05-18 00:30:30,viewed product,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4886f805,2018-05-18 00:07:23,search engine hit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Google,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4886f805,2018-05-18 00:11:56,checkout,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4886f805,2018-05-18 00:11:35,viewed product,NaN,9287.0,Samsung Galaxy J7 Prime,Muito Bom,32GB,Dourado,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
test= test[features]
test['label'] = 0 
test.head()

,event,model,condition,storage,color,search_term,campaign_source,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,label
person,,,,,,,,,,,,,,,,
4886f805,viewed product,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4886f805,viewed product,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4886f805,search engine hit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4886f805,checkout,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4886f805,viewed product,Samsung Galaxy J7 Prime,Muito Bom,32GB,Dourado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [19]:
test['event'] = lb_make.fit_transform(test['event'].astype(str))
test['model'] = lb_make.fit_transform(test['model'].astype(str))
test['condition'] = lb_make.fit_transform(test['condition'].astype(str))
test['color'] = lb_make.fit_transform(test['color'].astype(str))
test['search_term'] = lb_make.fit_transform(test['search_term'].astype(str))
test['campaign_source'] = lb_make.fit_transform(test['campaign_source'].astype(str))
test['channel'] = lb_make.fit_transform(test['channel'].astype(str))
test['storage'] = lb_make.fit_transform(test['storage'].astype(str))
test['new_vs_returning'] = lb_make.fit_transform(test['new_vs_returning'].astype(str))
test['city'] = lb_make.fit_transform(test['city'].astype(str))
test['region'] = lb_make.fit_transform(test['region'].astype(str))
test['country'] = lb_make.fit_transform(test['country'].astype(str))
test['device_type'] = lb_make.fit_transform(test['device_type'].astype(str))
test['screen_resolution'] = lb_make.fit_transform(test['screen_resolution'].astype(str))
test['operating_system_version'] = lb_make.fit_transform(test['operating_system_version'].astype(str))

In [20]:
test = test[features]
prediccion_final=model.predict(test)
prediccion_final

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [22]:
test['label'] = prediccion_final
test.head()

,event,model,condition,storage,color,search_term,campaign_source,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,label
person,,,,,,,,,,,,,,,,
4886f805,9,100,2,3,28,5675,16,7,2,1799,95,37,4,268,110,0
4886f805,9,100,2,3,28,5675,16,7,2,1799,95,37,4,268,110,0
4886f805,6,202,5,8,62,5675,16,7,2,1799,95,37,4,268,110,0
4886f805,2,100,2,3,28,5675,16,7,2,1799,95,37,4,268,110,0
4886f805,9,100,3,3,28,5675,16,7,2,1799,95,37,4,268,110,0


In [23]:
test = test.reset_index().drop_duplicates(subset='person', keep='last').set_index('person')
test.head()

,event,model,condition,storage,color,search_term,campaign_source,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,label
person,,,,,,,,,,,,,,,,
4886f805,10,202,5,8,62,5675,16,2,0,330,77,6,1,169,23,0
0297fc1e,5,192,5,8,62,5675,16,7,2,1799,95,37,4,268,110,0
2d681dd8,10,202,5,8,62,5675,16,3,1,877,81,6,0,55,69,0
cccea85e,10,202,5,8,62,5675,16,3,1,1147,81,6,0,83,67,0
4c8a8b93,10,202,5,8,62,5675,16,4,1,1487,81,6,1,169,23,0


In [24]:
test.drop(features, axis=1, inplace=True )


In [25]:
test.reset_index()

,person,label
0,4886f805,0
1,0297fc1e,0
2,2d681dd8,0
3,cccea85e,0
4,4c8a8b93,0
5,29ebb414,0
6,3dc1950f,0
7,8ea4c165,0
8,d8cfe234,0
9,d6bc64df,0


In [26]:
test.to_csv("../knn_n=20.csv")